In [13]:
''' Laufzeit grob 1.5h 
Average Precision@10: 0.41
Average Recall@10: 0.00'''

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            10083, 10084, 10085, 10086, 10087, 10088, 10089, 10090, 10091,
            10092],
           dtype='int64', length=10093)

# Task 2 : Extend your framework with audio-based retrieval systems and with evaluation metrics

### Team E

**Don't forget to update the version number after making changes** 

Version 2.0.2
Date: 11/12/2023

## Import Libraries  

In [1]:
import numpy as np
import pandas as pd
import json 


In [2]:
# import utility functions 
from ret import read_data # utility func to load data
from ret import get_id_from_info # utility func to return id by entering song's info
from ret import display_res # utility func to display results 
from ret import get_genre #utility func to get the genres from a list of id´s
from ret import get_genre_from_query #utility func to get the id and genre from the query
from ret import get_genre_from_ids #utility func to get the id and genre from the retrieved results

# import the retrieval systems 
from ret import random_baseline # baseline retrieval system that returns random results 
from ret import text_based # modularized text based retrieval system
from ret import audio_based # modularized audio based retrieval system

# import wrapper function to calculate cosine similarity
from ret import cos_sim 

# import evaluation functions 
from ret import gen_cov_10
from ret import ndcg_score
from ret import gen_div_10
from ret import calculate_precision_at_k
from ret import calculate_recall_at_k_vectorized
from ret import plot_precision_recall_curve
from ret import get_avg_recall_at_k
from ret import get_avg_precision_at_k

In [3]:
#!pip install sklearn

## Load Data

In [4]:
"""
Please put data files in ""./data/" before use 
"""

info = read_data("information")
genres = read_data("genres")

# text embeddings 
tfidf = read_data("lyrics_tf-idf")
word2vec = read_data("lyrics_word2vec")
bert = read_data("lyrics_bert")

# audio embeddings
blf_correlation = read_data("blf_correlation")
ivec256 = read_data("ivec256")
mfcc_stats = read_data("mfcc_stats")
musicnn = read_data("musicnn")

In [5]:
blf_correlation.drop(7298, axis=0, inplace=True)
genres.drop(7298, axis=0, inplace=True)

In [6]:
# Konvertieren Sie den DataFrame in ein NumPy-ndarray
blf_correlation_array = blf_correlation.iloc[:, 2:].to_numpy()
# Konvertieren Sie das NumPy-ndarray in eine NumPy-Matrix
blf_correlation_matrix = np.matrix(blf_correlation_array)


# Transponieren Sie die Matrix mit der transpose() Methode
blf_correlation_transpose = blf_correlation_matrix.transpose()

# Überprüfen Sie die Form der transponierten Matrizen
print(blf_correlation_transpose.shape) # (1328, 10093)
#print(len(blf_correlation), len(blf_correlation_transpose_zip[0])) # (1328, 10093)
# Berechnen Sie die Kosinusähnlichkeitsmatrix
cos_sim_matrix = np.triu(blf_correlation_matrix.dot(blf_correlation_transpose) / (np.linalg.norm(blf_correlation_matrix, axis=1) * np.linalg.norm(blf_correlation_matrix, axis=1)[:, None]))
# Fill the lower triangular part with the same values as the upper triangular part
cos_sim_matrix = cos_sim_matrix + cos_sim_matrix.T - np.diag(cos_sim_matrix.diagonal())

# Create a dataframe from the numpy matrix
df = pd.DataFrame(cos_sim_matrix)

# Assign the ids to the index and columns
df.index = blf_correlation.iloc[:, 0]
#df.index = df.index.str.strip('(),')
df.columns = blf_correlation.iloc[:, 0]
#df.columns = df.columns.str.strip('(),')

cos_sim_matrix

(1325, 10093)


array([[1.        , 0.63489262, 0.6353494 , ..., 0.48916107, 0.53107856,
        0.64144186],
       [0.63489262, 1.        , 0.82856014, ..., 0.69606716, 0.85213451,
        0.821675  ],
       [0.6353494 , 0.82856014, 1.        , ..., 0.69512885, 0.73822328,
        0.80549606],
       ...,
       [0.48916107, 0.69606716, 0.69512885, ..., 1.        , 0.6878554 ,
        0.68902471],
       [0.53107856, 0.85213451, 0.73822328, ..., 0.6878554 , 1.        ,
        0.79572456],
       [0.64144186, 0.821675  , 0.80549606, ..., 0.68902471, 0.79572456,
        1.        ]])

In [7]:
df = pd.merge(df, genres, on='id')

In [8]:
def change_to_set(s):
    # Wandeln Sie den String in eine Liste von Genres um
    l = s.strip("[]").split(",")
    # Entfernen Sie die Anführungszeichen um jedes Genre
    l = [g.strip("'") for g in l]
    # Wandeln Sie die Liste in ein Set um
    s = set(l)
    return s


In [9]:
def change_list_to_sets(list):
    array = []
    # Iterieren Sie über die Spalte in Ihrem DataFrame
    for s in list:
        s =change_to_set(s)
        # Fügen Sie das Set zum Array hinzu
        array.append(s)
    return array

In [10]:
#df = pd.merge(df, genres, on='id')
# Erstellen Sie ein leeres Array aus Sets
def get_relevancy_count(genre_query,genre_other_songs):
    array = change_list_to_sets(genre_other_songs)

    #print("Das Array aus Sets ist fertig")
    #print(array)



    # Initialize a counter
    count = 0

    print("Loop through the list of sets")
    for s in array:
        # Check if the intersection is not empty
        if genre_query.intersection(s):#
        # Increment the counter
            count += 1

    print("done")
    #print(count) # 1
    return count

In [11]:
df


,id,01Yfj2T3YTwJ1Yfy,01gyRHLquwXDlhkO,01rMxQv6vhyE1oQX,02RGE9FNH65RtMS7,02ZnlCGZEbkfCDxo,04OjszRi9rC5BlHC,04iitW3ffa0mhpx3,04xUDjAYC14jsHyH,06HvNTU9M9lnH71I,...,zxYtSeZzEVgPczJz,zxlnGZoud2KCmSaw,zyxOCKcXX1RmWpm0,zyz0UbYN4n9rHXex,zyzILCQvVeUFIINi,zzgS4ZqyswamEWNj,zzoFYDMlqU1X2zz1,zzpkRCGA5ud8q4mv,zzx8CWdM7qkxKQpC,genre
0,01Yfj2T3YTwJ1Yfy,1.000000,0.634893,0.635349,0.598153,0.576161,0.568466,0.629246,0.645637,0.587046,...,0.637629,0.654952,0.638904,0.560101,0.666608,0.611436,0.489161,0.531079,0.641442,"['rock', 'christian rock']"
1,01gyRHLquwXDlhkO,0.634893,1.000000,0.828560,0.851030,0.777809,0.686822,0.877492,0.708623,0.753095,...,0.823651,0.867349,0.869174,0.902670,0.652415,0.807221,0.696067,0.852135,0.821675,"['hip hop', 'rap', 'grindcore', 'death metal']"
2,01rMxQv6vhyE1oQX,0.635349,0.828560,1.000000,0.773646,0.761471,0.722970,0.869319,0.759349,0.774074,...,0.797611,0.805425,0.815107,0.764606,0.701674,0.761847,0.695129,0.738223,0.805496,"['rock', 'pop punk']"
3,02RGE9FNH65RtMS7,0.598153,0.851030,0.773646,1.000000,0.785102,0.664442,0.832017,0.627340,0.719576,...,0.819604,0.820042,0.844789,0.847214,0.609150,0.787245,0.708434,0.835343,0.749437,"['trance', 'techno', 'progressive trance']"
4,02ZnlCGZEbkfCDxo,0.576161,0.777809,0.761471,0.785102,1.000000,0.680657,0.820742,0.648261,0.731629,...,0.790634,0.768481,0.838606,0.745495,0.597407,0.798870,0.711728,0.768490,0.776015,"['pop', 'italian pop', 'latin', 'europop', 'am..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10088,zyzILCQvVeUFIINi,0.666608,0.652415,0.701674,0.609150,0.597407,0.567506,0.654330,0.653536,0.594052,...,0.636745,0.630093,0.634828,0.570979,1.000000,0.602570,0.503714,0.545268,0.670342,"['rock', 'pop', 'indie rock', 'folk rock', 'ne..."
10089,zzgS4ZqyswamEWNj,0.611436,0.807221,0.761847,0.787245,0.798870,0.669773,0.834442,0.660964,0.755063,...,0.769887,0.795303,0.855147,0.808174,0.602570,1.000000,0.749721,0.815573,0.818467,"['pop', 'rock', 'teen pop', 'soundtrack', 'sin..."
10090,zzoFYDMlqU1X2zz1,0.489161,0.696067,0.695129,0.708434,0.711728,0.670488,0.741595,0.590277,0.728201,...,0.705924,0.681825,0.741768,0.665708,0.503714,0.749721,1.000000,0.687855,0.689025,"['jazz', 'jazz fusion']"
10091,zzpkRCGA5ud8q4mv,0.531079,0.852135,0.738223,0.835343,0.768490,0.612901,0.826779,0.599824,0.700431,...,0.756766,0.829616,0.846157,0.864387,0.545268,0.815573,0.687855,1.000000,0.795725,"['soul', 'blues', 'r b', 'blues rock', 'southe..."


In [12]:

# Define the value of k
k = 10

# Define an empty list to store the precision and recall scores
precision_scores = []
recall_scores = []

# Loop over each query track
for query in df.id:
    print(query)
    #print("Sort the other tracks by their similarity scores in descending order and get genres")
    ranked_list = df.sort_values(by=query, ascending=False).genre.to_list()
    
    #print("Select the top k tracks from the ranked list")
    retrieved_list = ranked_list[1:k+1]
    #print(retrieved_list)
    
    # Get the genre of the query track
    query_genre = genres[genres['id'] == query]['genre'].iloc[0]
    #print(query_genre)
    query_genre = change_to_set(query_genre)
    #print(query_genre)
    
    #get genres  as set of top k tracks
    top_k_genre = change_list_to_sets(retrieved_list)
    #print(top_k_genre)
    #print("get count of all relevant tracks in dataset")
    all_relevant = get_relevancy_count(query_genre,df['genre'])
    #print("get count of all relevant tracks in top k")
    k_relevant = get_relevancy_count(query_genre,retrieved_list)
    
    recall =  k_relevant / all_relevant
    print(recall)
    
    precision = k_relevant/k
    print(precision)

    

    
    # Append the precision and recall scores to the list
    precision_scores.append(precision)
    recall_scores.append(recall)

# Calculate the average precision and recall over all query tracks
avg_precision = np.mean(precision_scores)
avg_recall = np.mean(recall_scores)

# Print the results
print(f'Average Precision@{k}: {avg_precision:.2f}')
print(f'Average Recall@{k}: {avg_recall:.2f}')


01Yfj2T3YTwJ1Yfy
Loop through the list of sets
done
Loop through the list of sets
done
0.0008453085376162299
0.1
01gyRHLquwXDlhkO
Loop through the list of sets
done
Loop through the list of sets
done
0.0025284450063211127
0.2
01rMxQv6vhyE1oQX
Loop through the list of sets
done
Loop through the list of sets
done
0.0007183908045977011
0.1
02RGE9FNH65RtMS7
Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
02ZnlCGZEbkfCDxo
Loop through the list of sets
done
Loop through the list of sets
done
0.0011366865586814436
0.4
04OjszRi9rC5BlHC
Loop through the list of sets
done
Loop through the list of sets
done
0.0012279355333844973
0.8
04iitW3ffa0mhpx3
Loop through the list of sets
done
Loop through the list of sets
done
0.001188707280832095
0.4
04xUDjAYC14jsHyH
Loop through the list of sets
done
Loop through the list of sets
done
0.0020394289598912306
0.3
06HvNTU9M9lnH71I
Loop through the list of sets
done
Loop through the list of sets
done
0.0019588638589618022
0.2
06

Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
0QSQ3CDT5x4GRIgi
Loop through the list of sets
done
Loop through the list of sets
done
0.0009649404953361209
0.6
0QSuci5nyuRojGbo
Loop through the list of sets
done
Loop through the list of sets
done
0.0020161290322580645
0.3
0RJXFR9AZUgJM9I0
Loop through the list of sets
done
Loop through the list of sets
done
0.00098015192354815
0.4
0RswMLgk4KS4GbIP
Loop through the list of sets
done
Loop through the list of sets
done
0.010033444816053512
0.3
0SJ5yUTHjKiPaB3z
Loop through the list of sets
done
Loop through the list of sets
done
0.0010145417653026716
0.3
0SpykgFTUzfUTJkN
Loop through the list of sets
done
Loop through the list of sets
done
0.0009242144177449168
0.5
0TanEa2QCnKAedsL
Loop through the list of sets
done
Loop through the list of sets
done
0.014
0.7
0Tw7xPCFY8Kkznev
Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
0UHzuflmmfXZkZNQ
Loop through the list of sets
done
Loo

Loop through the list of sets
done
Loop through the list of sets
done
0.0007320644216691069
0.1
0qp2KyXH2ustSNci
Loop through the list of sets
done
Loop through the list of sets
done
0.001183431952662722
0.9
0r6fQYlWtk4HkN1F
Loop through the list of sets
done
Loop through the list of sets
done
0.0009656237929702588
0.5
0rglHs2VvXylYUjI
Loop through the list of sets
done
Loop through the list of sets
done
0.0011284862163469288
0.7
0rmouv9XbAHY2XBZ
Loop through the list of sets
done
Loop through the list of sets
done
0.001976284584980237
1.0
0roDBBFUmgkbD7Zg
Loop through the list of sets
done
Loop through the list of sets
done
0.0010570824524312897
0.6
0rplhtI8mUMSa59h
Loop through the list of sets
done
Loop through the list of sets
done
0.0028089887640449437
0.5
0sgehyyNEKd2ar50
Loop through the list of sets
done
Loop through the list of sets
done
0.0014762326542663124
0.5
0swSt84vjuuHE3qZ
Loop through the list of sets
done
Loop through the list of sets
done
0.0006983240223463687
0.4
0t

Loop through the list of sets
done
Loop through the list of sets
done
0.0013917884481558804
0.6
1GX3zPl6DaMjCoYx
Loop through the list of sets
done
Loop through the list of sets
done
0.00025940337224383917
0.1
1GngSVshm9HPaMVa
Loop through the list of sets
done
Loop through the list of sets
done
0.001101494885916601
0.7
1GxSKmrbSnZvDIJO
Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
1H7yQe1NljCbMapi
Loop through the list of sets
done
Loop through the list of sets
done
0.002116153303550435
0.9
1HJwHtefKWOKO7Fo
Loop through the list of sets
done
Loop through the list of sets
done
0.0012621709340064912
0.7
1HNbB2jYOPVyrBwZ
Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
1HmXKsTebNB1Uz4u
Loop through the list of sets
done
Loop through the list of sets
done
0.0012667390517553383
0.7
1IBHNIcbKXId8a0X
Loop through the list of sets
done
Loop through the list of sets
done
0.0009306654257794323
0.4
1IPmO9HuMPFb7pBs
Loop through the lis

Loop through the list of sets
done
Loop through the list of sets
done
0.0033112582781456954
0.5
1cKGmPvdSdGq8jg9
Loop through the list of sets
done
Loop through the list of sets
done
0.0027397260273972603
0.6
1cUUnlwxUH9rXdBd
Loop through the list of sets
done
Loop through the list of sets
done
0.004477611940298508
0.6
1cyfUPFQk1JSdRRw
Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
1d0rrZRbKBwrZZgz
Loop through the list of sets
done
Loop through the list of sets
done
0.0011352885525070956
0.6
1dRg09BK9AaMKXDp
Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
1desuKPRoCCWc5xZ
Loop through the list of sets
done
Loop through the list of sets
done
0.0016999575010624734
0.4
1eWUxY3hOm6M3Szp
Loop through the list of sets
done
Loop through the list of sets
done
0.0011206574523720584
0.6
1f13MAfipcAOI0Ju
Loop through the list of sets
done
Loop through the list of sets
done
0.008403361344537815
0.1
1fDah5r1pTHBacmr
Loop through the list

...

Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
xathVhYc6pR34CqP
Loop through the list of sets
done
Loop through the list of sets
done
0.0010812480691998764
0.7
xavjaotTVboQdTEj
Loop through the list of sets
done
Loop through the list of sets
done
0.0014104372355430183
0.8
xaxkA9i6lpSv1WtM
Loop through the list of sets
done
Loop through the list of sets
done
0.0004677268475210477
0.1
xbPROldsip73RSkA
Loop through the list of sets
done
Loop through the list of sets
done
0.005372733378106112
0.8
xbjfiDE5tCYP0iYT
Loop through the list of sets
done
Loop through the list of sets
done
0.0048543689320388345
0.2
xbwLrzoPbiywa28c
Loop through the list of sets
done
Loop through the list of sets
done
0.001669449081803005
0.2
xcRMLQyZmL7tnKoA
Loop through the list of sets
done
Loop through the list of sets
done
0.000877963125548727
0.3
xcWYLAbgy0udCFmC
Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
xczlK5poIY8HcEc7
Loop through the list 

Loop through the list of sets
done
Loop through the list of sets
done
0.002296211251435132
0.2
y6nm14TvZDNrdBbc
Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
y7YlVS4ptVmDNpCe
Loop through the list of sets
done
Loop through the list of sets
done
0.005111821086261981
0.8
y7wjEOq4a6FnwhXu
Loop through the list of sets
done
Loop through the list of sets
done
0.009345794392523364
0.1
y8WknAhGzZh4D24p
Loop through the list of sets
done
Loop through the list of sets
done
0.0012670256572695597
0.4
y8lmwipz9oldFxS5
Loop through the list of sets
done
Loop through the list of sets
done
0.0055658627087198514
0.3
y8n0jmGDTveNeIt0
Loop through the list of sets
done
Loop through the list of sets
done
0.001015400236926722
0.6
y8wp2cUBzIEYsouc
Loop through the list of sets
done
Loop through the list of sets
done
0.0032817627754336614
0.7
y9SEGMu4E3iUNvqA
Loop through the list of sets
done
Loop through the list of sets
done
0.00018304960644334616
0.1
y9ffMY2ajzA6ehgR
Loop

Loop through the list of sets
done
Loop through the list of sets
done
0.0011853959222380276
0.5
ybykwswgftxuoUP2
Loop through the list of sets
done
Loop through the list of sets
done
0.0012779552715654952
0.6
ycigyHRUCEoBuEUO
Loop through the list of sets
done
Loop through the list of sets
done
0.0007572889057175312
0.4
yd9BOQS0uYWbeQVu
Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
ydLADICszvB61OtR
Loop through the list of sets
done
Loop through the list of sets
done
0.0009950248756218905
0.6
ydS4NguMgrD2NgdG
Loop through the list of sets
done
Loop through the list of sets
done
0.0012818458580355712
0.8
yeTAY1wfeosSJh8D
Loop through the list of sets
done
Loop through the list of sets
done
0.0013736263736263737
0.8
yf4lHIy5eaOp9U2I
Loop through the list of sets
done
Loop through the list of sets
done
0.0006361323155216285
0.2
yfH1SiGbrZw9qZqD
Loop through the list of sets
done
Loop through the list of sets
done
0.0006062443164595332
0.2
yfpVrCQX2Q0SoWFE
L

Loop through the list of sets
done
Loop through the list of sets
done
0.0010884353741496598
0.4
z4iZWTSpd6orQtJe
Loop through the list of sets
done
Loop through the list of sets
done
0.0
0.0
z4pfNCH1xUBerW1h
Loop through the list of sets
done
Loop through the list of sets
done
0.0005592841163310962
0.3
z5JjR5elJdSz32i8
Loop through the list of sets
done
Loop through the list of sets
done
0.0012628920227320563
0.6
z5qGwH461rz6ndPE
Loop through the list of sets
done
Loop through the list of sets
done
0.00847457627118644
0.1
z6R8nQvSwnGvQO4M
Loop through the list of sets
done
Loop through the list of sets
done
0.006369426751592357
0.1
z6j688GNboKT3CIN
Loop through the list of sets
done
Loop through the list of sets
done
0.005393743257820928
0.5
z6v87WSfjQK8bNKv
Loop through the list of sets
done
Loop through the list of sets
done
0.0006319780914261639
0.3
z7L9lXo1gIFl7L6C
Loop through the list of sets
done
Loop through the list of sets
done
0.0009621552277100705
0.3
zA5krYa5HpkeYoux
Loop 

Loop through the list of sets
done
Loop through the list of sets
done
0.001442221020371372
0.8
zbI3TrMSY2chqzRz
Loop through the list of sets
done
Loop through the list of sets
done
0.0009671179883945841
0.6
zc0de8jOQzVtc0fN
Loop through the list of sets
done
Loop through the list of sets
done
0.0006077180188392585
0.2
zcRqATRcssHILwXO
Loop through the list of sets
done
Loop through the list of sets
done
0.001283491095780523
0.8
ze51XBGDaJjeAbEn
Loop through the list of sets
done
Loop through the list of sets
done
0.0014684287812041115
0.1
zeKjmKHNDeh1XT7d
Loop through the list of sets
done
Loop through the list of sets
done
0.0014917024053701287
0.8
zeoIIrQqqA7dMJ7s
Loop through the list of sets
done
Loop through the list of sets
done
0.0012706480304955528
0.7
zf46iiijsjRmgLzE
Loop through the list of sets
done
Loop through the list of sets
done
0.00102880658436214
0.6
zf6cXW4e4URhsNmb
Loop through the list of sets
done
Loop through the list of sets
done
0.0014681592952835382
0.8
zfVN

In [14]:
index_value

NameError: name 'index_value' is not defined

In [ ]:
df = pd.merge(df, genres, on='id')

In [ ]:
ranked_list = df.sort_values(by='zyzILCQvVeUFIINi', ascending=False).genre.tolist()

In [ ]:
ranked_list

In [ ]:
# Die list comprehension, die jeden String in der Spalte genre in ein Set umwandelt und zum Array hinzufügt
array = [set(df.genre[i].split(",")[0][2:], df.genre[i].split(",")[-1][:-2]) for i in range(len(df.genre))]

# Das Array aus Sets ausgeben
print(array)

In [ ]:
# Erstellen Sie ein leeres Array aus Sets
array = []

# Iterieren Sie über die Spalte in Ihrem DataFrame
for s in df['genre']:
  # Wandeln Sie den String in eine Liste von Genres um
  l = s.strip("[]").split(",")
  # Entfernen Sie die Anführungszeichen um jedes Genre
  l = [g.strip("'") for g in l]
  # Wandeln Sie die Liste in ein Set um
  s = set(l)
  # Fügen Sie das Set zum Array hinzu
  array.append(s)

# Das Array aus Sets ist fertig
print(array)

In [ ]:
l

In [ ]:
type(df.index[1])


In [ ]:
get_avg_recall_at_k(blf_correlation, 10, genres)

In [ ]:
type(genres["id"][1])

In [ ]:
genres[genres["id"]== "02RGE9FNH65RtMS7"]


In [ ]:
# Import numpy
import numpy as np

# Define a sample repr data set with 5 vectors, one of them having a zero norm
repr = blf_correlation
repr = repr.iloc[:, 2:].to_numpy()

# Calculate the norm of each vector along the rows
norms = np.linalg.norm(repr, axis=1)

# Create a boolean mask of the condition that the norm is zero
mask = np.isclose(norms, 0)

# Use numpy.nonzero to find the indices of the vectors that have a zero norm
indices = np.nonzero(mask)

# Use numpy indexing to get the problematic vectors
problematic_vectors = repr[indices]

# Print the indices and the problematic vectors
print("The indices of the problematic vectors are:", indices)
print("The problematic vectors are:", problematic_vectors)


In [ ]:
"""
Initialize empty dictionary to store the results

User guide:
the dictionary has three dimensions:
1st dim: query tracks 
available tags ["track1", "track2", "track3"]
2nd dim: retrieval systems 
available tags ["base_line", "tfidf", "word2vec", "bert", "blf_correlation", "ivec256", "mfcc_stats", "musicnn"]
3rd dim: evaluation metrics and retrieved tracks 
available tags ["tracks", "precision@10", "recall@10", "genre_diversity@10", "genre_coverage@10", "ndcg"]

Example:
res["track1"]["base_line"]["tracks"]

"""
with open('result_mod.json', 'r') as f:
    res = json.load(f)

## Track 1 
"Love Me" "The 1975"

In [ ]:
name = "Love Me"
artist = "The 1975"
id_track1 = get_id_from_info(name, artist, info)

In [ ]:
# retrieve tracks with random baseline 
res["track1"]["base_line"]["tracks"] = random_baseline(id=id_track1, info=info, N=10)

In [ ]:
# retrieve tracks with text based retrieval systems 
res["track1"]["tfidf"]["tracks"] = text_based(id=id_track1, repr=tfidf, N=10, sim_func=cos_sim)
res["track1"]["word2vec"]["tracks"] = text_based(id=id_track1, repr=word2vec, N=10, sim_func=cos_sim)
res["track1"]["bert"]["tracks"] = text_based(id=id_track1, repr=bert, N=10, sim_func=cos_sim)

In [ ]:
# retrieve tracks with audio based retrieval systems 
res["track1"]["blf_correlation"]["tracks"] = audio_based(id=id_track1, repr=blf_correlation, N=10, sim_func=cos_sim)
print("blf correlation Results:")
display_res(res["track1"]["blf_correlation"]["tracks"], info)

res["track1"]["ivec256"]["tracks"] = audio_based(id=id_track1, repr=ivec256, N=10, sim_func=cos_sim)
print("ivec256 Results:")
display_res(res["track1"]["ivec256"]["tracks"], info)

res["track1"]["mfcc_stats"]["tracks"] = audio_based(id=id_track1, repr=mfcc_stats, N=10, sim_func=cos_sim)
print("MFCC Stats Results:")
display_res(res["track1"]["mfcc_stats"]["tracks"], info)

res["track1"]["musicnn"]["tracks"] = audio_based(id=id_track1, repr=musicnn, N=10, sim_func=cos_sim)
print("Musicnn Results:")
display_res(res["track1"]["musicnn"]["tracks"], info)

In [ ]:
#retrieve genres of the results for precision@10 & recall@10 calculation
q1_genres_mfcc_stats = get_genre_from_ids(res["track1"]["mfcc_stats"]["tracks"],genres)
q1_genres_blf_correlation = get_genre_from_ids(res["track1"]["blf_correlation"]["tracks"], genres)
q1_genres_ivec256 = get_genre_from_ids(res["track1"]["ivec256"]["tracks"], genres)
q1_genres_musicnn = get_genre_from_ids(res["track1"]["musicnn"]["tracks"], genres)
q1_genres_tfidf = get_genre_from_ids(res["track1"]["tfidf"]["tracks"], genres)
q1_genres_word2vec = get_genre_from_ids(res["track1"]["word2vec"]["tracks"],genres)
q1_genres_bert =  get_genre_from_ids(res["track1"]["bert"]["tracks"],genres)
q1_genres_base_line = get_genre_from_ids(res["track1"]["base_line"]["tracks"],genres)

In [ ]:
# calculate precision @10 for track1

#get query genre
query_genre1 = get_genre_from_query(id_track1, genres)

#query track 1 precision (k=10) for all 8 Retrieval Systems
res["track1"]["mfcc_stats"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_mfcc_stats,10)
precision = res["track1"]["mfcc_stats"]["precision@10"]
print(f"precision@10: {precision}")

res["track1"]["blf_correlation"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_blf_correlation,10)
precision = res["track1"]["blf_correlation"]["precision@10"]
print(f"precision@10: {precision}")

res["track1"]["ivec256"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_ivec256,10)
precision = res["track1"]["ivec256"]["precision@10"] 
print(f"precision@10: {precision}")

res["track1"]["musicnn"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_musicnn,10)
precision = res["track1"]["musicnn"]["precision@10"]
print(f"precision@10: {precision}")

res["track1"]["tfidf"]["precision@10"]= calculate_precision_at_k(query_genre1,q1_genres_tfidf,10)
precision = res["track1"]["tfidf"]["precision@10"]
print(f"precision@10: {precision}")

res["track1"]["word2vec"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_word2vec,10)
precision = res["track1"]["word2vec"]["precision@10"]
print(f"precision@10: {precision}")

res["track1"]["bert"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_bert,10)
precision = res["track1"]["bert"]["precision@10"] 
print(f"precision@10: {precision}")

res["track1"]["base_line"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_base_line,10)
precision = res["track1"]["base_line"]["precision@10"]
print(f"precision@10: {precision}")

In [ ]:
genres_list = genres.values.tolist()
res["track1"]["mfcc_stats"]["recall@10"] = calculate_recall_at_k_vectorized(query_genre1, q1_genres_mfcc_stats,genres_list,10)
recall = res["track1"]["mfcc_stats"]["recall@10"]
print(f"recall@10: {recall}")

In [ ]:
genres.values.tolist()


In [ ]:
#query track 1 recall (k=10) for all 8 Retrieval Systems
#create genres dataset list for later use


res["track1"]["mfcc_stats"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_mfcc_stats,genres_list,10)
recall = res["track1"]["mfcc_stats"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["blf_correlation"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_blf_correlation,genres_list,10)
recall = res["track1"]["blf_correlation"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["ivec256"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_ivec256,genres_list,10)
recall = res["track1"]["ivec256"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["musicnn"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_musicnn,genres_list,10)
recall = res["track1"]["musicnn"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["tfidf"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_tfidf,genres_list,10)
recall = res["track1"]["tfidf"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["word2vec"]["recall@10"]= calculate_recall_at_k(query_genre1, q1_genres_word2vec,genres_list,10)
recall = res["track1"]["word2vec"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["bert"]["recall@10"]= calculate_recall_at_k(query_genre1,  q1_genres_bert,genres_list,10)
recall = res["track1"]["bert"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["base_line"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_base_line,genres_list,10)
recall = res["track1"]["base_line"]["recall@10"]
print(f"recall@10: {recall}")

In [ ]:
#retrieve 100 results for precision-recall plot (audio based) because k should be varied in the interval [1,100]
q1_100_mfcc_stats = get_genre_from_ids(audio_based(id=id_track1, repr=mfcc_stats, N=100, sim_func=cos_sim), genres)
q1_100_blf_correlation = get_genre_from_ids(audio_based(id=id_track1, repr=blf_correlation, N=100, sim_func=cos_sim), genres)
q1_100_ivec256 = get_genre_from_ids(audio_based(id=id_track1, repr=ivec256, N=100, sim_func=cos_sim), genres)
q1_100_musicnn = get_genre_from_ids(audio_based(id=id_track1, repr=musicnn, N=100, sim_func=cos_sim), genres)
#retrieve 100 text based results for precision recall plot because k should be varied in the interval [1,100]
q1_100_tfidf = get_genre_from_ids(text_based(id=id_track1, repr=tfidf, N=100, sim_func=cos_sim), genres)
q1_100_word2vec = get_genre_from_ids(text_based(id=id_track1, repr=word2vec, N=100, sim_func=cos_sim),genres)
q1_100_bert =  get_genre_from_ids(text_based(id=id_track1, repr=bert, N=100, sim_func=cos_sim),genres)
q1_100_base_line = get_genre_from_ids(random_baseline(id=id_track1, info=info, N=100),genres)


#plot precision recall curve for query track1 for all evaluated systems

system_data1 = {
    "audio mfcc stats": {"system_name": "Audio MFCC Stats","query_genre": query_genre1, "retrieved_genres": q1_100_mfcc_stats, "dataset_genres": genres_list},
    "audio blf correlation": {"system_name": "Audio BLF Correlation","query_genre": query_genre1, "retrieved_genres": q1_100_blf_correlation, "dataset_genres": genres_list},
    "audio ivec_256": {"system_name": "Audio iVec 256","query_genre": query_genre1, "retrieved_genres": q1_100_ivec256, "dataset_genres":genres_list},
    "audio muiscnn": {"system_name": "Audio Musicnn","query_genre": query_genre1, "retrieved_genres": q1_100_musicnn, "dataset_genres":genres_list},
    "text tf-idf": {"system_name": "Text TF-IDF","query_genre": query_genre1, "retrieved_genres": q1_100_tfidf, "dataset_genres":genres_list},
    "text word2vec": {"system_name": "Text Word2Vec","query_genre": query_genre1, "retrieved_genres": q1_100_word2vec, "dataset_genres":genres_list},
    "text bert": {"system_name": "Text BERT","query_genre": query_genre1, "retrieved_genres": q1_genres_bert, "dataset_genres":genres_list},
    "text random": {"system_name": "Text Random Baseline","query_genre": query_genre1, "retrieved_genres": q1_100_base_line, "dataset_genres":genres_list},
}

plot_precision_recall_curve(system_data1)

In [ ]:
# calculate genre diversity@10
all_genres = list(set([item for id in genres['id'] for item in get_genre(id, genres)]))

#audio based
q1_genre_mfcc_stats = [get_genre(id, genres) for id in res["track1"]["mfcc_stats"]["tracks"]]
q1_genre_blf_correlation = [get_genre(id, genres) for id in res["track1"]["blf_correlation"]["tracks"]]
q1_genre_ivec256 = [get_genre(id, genres) for id in res["track1"]["ivec256"]["tracks"]]
q1_genre_musicnn = [get_genre(id, genres) for id in res["track1"]["musicnn"]["tracks"]]
#text based
q1_genre_tfidf = [get_genre(id, genres) for id in res["track1"]["tfidf"]["tracks"]]
q1_genre_word2vec = [get_genre(id, genres) for id in res["track1"]["word2vec"]["tracks"]]
q1_genre_bert = [get_genre(id, genres) for id in res["track1"]["bert"]["tracks"]]
q1_genre_base_line = [get_genre(id, genres) for id in res["track1"]["base_line"]["tracks"]]

#calculate genre diversity @10
res["track1"]["mfcc_stats"]["genre_diversity@10"] = gen_div_10(q1_genre_mfcc_stats, all_genres, N=10)
genre_div = res["track1"]["mfcc_stats"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["blf_correlation"]["genre_diversity@10"] = gen_div_10(q1_genre_blf_correlation,all_genres,N=10)
genre_div = res["track1"]["blf_correlation"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["ivec256"]["genre_diversity@10"] = gen_div_10(q1_genre_ivec256,all_genres,N=10)
genre_div = res["track1"]["ivec256"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["musicnn"]["genre_diversity@10"] = gen_div_10(q1_genre_musicnn,all_genres,N=10)
genre_div = res["track1"]["musicnn"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["tfidf"]["genre_diversity@10"] = gen_div_10(q1_genre_tfidf,all_genres,N=10)
genre_div = res["track1"]["tfidf"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["word2vec"]["genre_diversity@10"] = gen_div_10(q1_genre_word2vec,all_genres,N=10)
genre_div = res["track1"]["word2vec"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["bert"]["genre_diversity@10"] = gen_div_10(q1_genre_bert,all_genres,N=10)
genre_div = res["track1"]["bert"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["base_line"]["genre_diversity@10"] = gen_div_10(q1_genre_base_line,all_genres,N=10)
genre_div = res["track1"]["base_line"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")


In [ ]:
print(query_genre1)

In [ ]:
# calculate genre coverage@10 

res["track1"]["mfcc_stats"]["genre_coverage@10"] = gen_cov_10(res["track1"]["mfcc_stats"]["tracks"], genres)
genre_cov = res["track1"]["mfcc_stats"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["blf_correlation"]["genre_coverage@10"] = gen_cov_10(res["track1"]["blf_correlation"]["tracks"], genres)
genre_cov = res["track1"]["blf_correlation"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["ivec256"]["genre_coverage@10"] = gen_cov_10(res["track1"]["ivec256"]["tracks"], genres)
genre_cov = res["track1"]["ivec256"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["musicnn"]["genre_coverage@10"] = gen_cov_10(res["track1"]["musicnn"]["tracks"], genres)
genre_cov = res["track1"]["musicnn"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["tfidf"]["genre_coverage@10"] = gen_cov_10(res["track1"]["tfidf"]["tracks"], genres)
genre_cov = res["track1"]["tfidf"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["word2vec"]["genre_coverage@10"] = gen_cov_10(res["track1"]["word2vec"]["tracks"], genres)
genre_cov = res["track1"]["word2vec"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["bert"]["genre_coverage@10"] = gen_cov_10(res["track1"]["bert"]["tracks"], genres)
genre_cov = res["track1"]["bert"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["base_line"]["genre_coverage@10"] = gen_cov_10(res["track1"]["base_line"]["tracks"], genres)
genre_cov = res["track1"]["base_line"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

In [ ]:
# calculate ndcg@10 

res["track1"]["mfcc_stats"]["ndcg"] = ndcg_score(id_track1, res["track1"]["mfcc_stats"]["tracks"], genres)
ndcg = res["track1"]["mfcc_stats"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["blf_correlation"]["ndcg"] = ndcg_score(id_track1, res["track1"]["blf_correlation"]["tracks"], genres)
ndcg = res["track1"]["blf_correlation"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["ivec256"]["ndcg"] = ndcg_score(id_track1, res["track1"]["ivec256"]["tracks"], genres)
ndcg = res["track1"]["ivec256"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["musicnn"]["ndcg"] = ndcg_score(id_track1, res["track1"]["musicnn"]["tracks"], genres)
ndcg = res["track1"]["musicnn"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["tfidf"]["ndcg"] = ndcg_score(id_track1, res["track1"]["tfidf"]["tracks"], genres)
ndcg = res["track1"]["tfidf"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["word2vec"]["ndcg"] = ndcg_score(id_track1, res["track1"]["word2vec"]["tracks"], genres)
ndcg = res["track1"]["word2vec"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["bert"]["ndcg"] = ndcg_score(id_track1, res["track1"]["bert"]["tracks"], genres)
ndcg = res["track1"]["bert"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["base_line"]["ndcg"] = ndcg_score(id_track1, res["track1"]["base_line"]["tracks"], genres)
ndcg = res["track1"]["base_line"]["ndcg"]
print(f"ndcg: {ndcg}")

## Track 2 
"One" "U2"

In [ ]:
name = 'One'
artist = 'U2'
id_track2 = get_id_from_info(name, artist, info)

In [ ]:
# retrieve tracks with random baseline 
res["track2"]["base_line"]["tracks"] = random_baseline(id=id_track2, info=info, N=10)

In [ ]:
# retrieve tracks with text based retrieval systems 
res["track2"]["tfidf"]["tracks"] = text_based(id=id_track2, repr=tfidf, N=10, sim_func=cos_sim)
res["track2"]["word2vec"]["tracks"] = text_based(id=id_track2, repr=word2vec, N=10, sim_func=cos_sim)
res["track2"]["bert"]["tracks"] = text_based(id=id_track2, repr=bert, N=10, sim_func=cos_sim)

In [ ]:
# retrieve tracks with audio based retrieval systems 
res["track2"]["blf_correlation"]["tracks"] = audio_based(id=id_track2, repr=blf_correlation, N=10, sim_func=cos_sim)
print("blf correlation Results:")
display_res(res["track2"]["blf_correlation"]["tracks"], info)

res["track2"]["ivec256"]["tracks"] = audio_based(id=id_track2, repr=ivec256, N=10, sim_func=cos_sim)
print("ivec 256 Results:")
display_res(res["track2"]["ivec256"]["tracks"], info)

res["track2"]["mfcc_stats"]["tracks"] = audio_based(id=id_track2, repr=mfcc_stats, N=10, sim_func=cos_sim)
print("mfcc_stats Results:")
display_res(res["track2"]["mfcc_stats"]["tracks"], info)

res["track2"]["musicnn"]["tracks"] = audio_based(id=id_track2, repr=musicnn, N=10, sim_func=cos_sim)
print("musicnn Results:")
display_res(res["track2"]["musicnn"]["tracks"], info)

In [ ]:
#retrieve genres of the results for precision@10 & recall@10 calculation
q2_genres_mfcc_stats = get_genre_from_ids(res["track2"]["mfcc_stats"]["tracks"],genres)
q2_genres_blf_correlation = get_genre_from_ids(res["track2"]["blf_correlation"]["tracks"], genres)
q2_genres_ivec256 = get_genre_from_ids(res["track2"]["ivec256"]["tracks"], genres)
q2_genres_musicnn = get_genre_from_ids(res["track2"]["musicnn"]["tracks"], genres)
q2_genres_tfidf = get_genre_from_ids(res["track2"]["tfidf"]["tracks"], genres)
q2_genres_word2vec = get_genre_from_ids(res["track2"]["word2vec"]["tracks"],genres)
q2_genres_bert =  get_genre_from_ids(res["track2"]["bert"]["tracks"],genres)
q2_genres_base_line = get_genre_from_ids(res["track2"]["base_line"]["tracks"],genres)

In [ ]:
# calculate precision @10 for track2
#get query genre
query_genre2 = get_genre_from_query(id_track2, genres)

#query track 1 precision (k=10) for all 8 Retrieval Systems
res["track2"]["mfcc_stats"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_mfcc_stats,10)
precision = res["track2"]["mfcc_stats"]["precision@10"]
print(f"precision@10: {precision}")

res["track2"]["blf_correlation"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_blf_correlation,10)
precision = res["track2"]["blf_correlation"]["precision@10"]
print(f"precision@10: {precision}")

res["track2"]["ivec256"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_ivec256,10)
precision = res["track2"]["ivec256"]["precision@10"] 
print(f"precision@10: {precision}")

res["track2"]["musicnn"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_musicnn,10)
precision = res["track2"]["musicnn"]["precision@10"]
print(f"precision@10: {precision}")

res["track2"]["tfidf"]["precision@10"]= calculate_precision_at_k(query_genre2,q2_genres_tfidf,10)
precision = res["track2"]["tfidf"]["precision@10"]
print(f"precision@10: {precision}")

res["track2"]["word2vec"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_word2vec,10)
precision = res["track2"]["word2vec"]["precision@10"]
print(f"precision@10: {precision}")

res["track2"]["bert"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_bert,10)
precision = res["track2"]["bert"]["precision@10"] 
print(f"precision@10: {precision}")

res["track2"]["base_line"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_base_line,10)
precision = res["track2"]["base_line"]["precision@10"]
print(f"precision@10: {precision}")


In [ ]:
#query track 2 recall (k=10) for all 8 Retrieval Systems
#create genres dataset list for later use
genres_list = genres.values.tolist()

res["track2"]["mfcc_stats"]["recall@10"] = calculate_recall_at_k(query_genre2,q2_genres_mfcc_stats,genres_list,10)
recall = res["track2"]["mfcc_stats"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["blf_correlation"]["recall@10"] = calculate_recall_at_k(query_genre2,q2_genres_blf_correlation,genres_list,10)
recall = res["track2"]["blf_correlation"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["ivec256"]["recall@10"] = calculate_recall_at_k(query_genre2,q2_genres_ivec256,genres_list,10)
recall = res["track2"]["ivec256"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["musicnn"]["recall@10"] = calculate_recall_at_k(query_genre2, q2_genres_musicnn,genres_list,10)
recall = res["track2"]["musicnn"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["tfidf"]["recall@10"] = calculate_recall_at_k(query_genre2, q2_genres_tfidf,genres_list,10)
recall = res["track2"]["tfidf"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["word2vec"]["recall@10"]= calculate_recall_at_k(query_genre2, q2_genres_word2vec,genres_list,10)
recall = res["track2"]["word2vec"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["bert"]["recall@10"]= calculate_recall_at_k(query_genre2,q2_genres_bert,genres_list,10)
recall = res["track2"]["bert"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["base_line"]["recall@10"] = calculate_recall_at_k(query_genre2,q2_genres_base_line,genres_list,10)
recall = res["track2"]["base_line"]["recall@10"]
print(f"recall@10: {recall}")

In [ ]:
#retrieve 100 results for precision-recall plot (audio based) because k should be varied in the interval [1,100]
q2_100_mfcc_stats = get_genre_from_ids(audio_based(id=id_track2, repr=mfcc_stats, N=100, sim_func=cos_sim), genres)
q2_100_blf_correlation = get_genre_from_ids(audio_based(id=id_track2, repr=blf_correlation, N=100, sim_func=cos_sim), genres)
q2_100_ivec256 = get_genre_from_ids(audio_based(id=id_track2, repr=ivec256, N=100, sim_func=cos_sim), genres)
q2_100_musicnn = get_genre_from_ids(audio_based(id=id_track2, repr=musicnn, N=100, sim_func=cos_sim), genres)
q2_100_tfidf = get_genre_from_ids(text_based(id=id_track2, repr=tfidf, N=100, sim_func=cos_sim), genres)
q2_100_word2vec = get_genre_from_ids(text_based(id=id_track2, repr=word2vec, N=100, sim_func=cos_sim),genres)
q2_100_bert =  get_genre_from_ids(text_based(id=id_track2, repr=bert, N=100, sim_func=cos_sim),genres)
q2_100_base_line = get_genre_from_ids(random_baseline(id=id_track2, info=info, N=100),genres)

#plot precision recall curve for query track2 for all evaluated systems

system_data2 = {
    "audio mfcc stats": {"system_name": "Audio MFCC Stats","query_genre": query_genre2, "retrieved_genres": q2_100_mfcc_stats, "dataset_genres": genres_list},
    "audio blf correlation": {"system_name": "Audio BLF Correlation","query_genre": query_genre2, "retrieved_genres": q2_100_blf_correlation, "dataset_genres": genres_list},
    "audio ivec_256": {"system_name": "Audio iVec 256","query_genre": query_genre2, "retrieved_genres": q2_100_ivec256, "dataset_genres":genres_list},
    "audio muiscnn": {"system_name": "Audio Musicnn","query_genre": query_genre2, "retrieved_genres": q2_100_musicnn, "dataset_genres":genres_list},
    "text tf-idf": {"system_name": "Text TF-IDF","query_genre": query_genre2, "retrieved_genres": q2_100_tfidf, "dataset_genres":genres_list},
    "text word2vec": {"system_name": "Text Word2Vec","query_genre": query_genre2, "retrieved_genres": q2_100_word2vec, "dataset_genres":genres_list},
    "text bert": {"system_name": "Text BERT","query_genre": query_genre2, "retrieved_genres": q2_genres_bert, "dataset_genres":genres_list},
    "text random": {"system_name": "Text Random Baseline","query_genre": query_genre2, "retrieved_genres": q2_100_base_line, "dataset_genres":genres_list},
}

plot_precision_recall_curve(system_data2)

In [ ]:
# calculate genre diversity@10
all_genres = list(set([item for id in genres['id'] for item in get_genre(id, genres)]))

#audio based
q2_genre_mfcc_stats = [get_genre(id, genres) for id in res["track2"]["mfcc_stats"]["tracks"]]
q2_genre_blf_correlation = [get_genre(id, genres) for id in res["track2"]["blf_correlation"]["tracks"]]
q2_genre_ivec256 = [get_genre(id, genres) for id in res["track2"]["ivec256"]["tracks"]]
q2_genre_musicnn = [get_genre(id, genres) for id in res["track2"]["musicnn"]["tracks"]]
#text based
q2_genre_tfidf = [get_genre(id, genres) for id in res["track2"]["tfidf"]["tracks"]]
q2_genre_word2vec = [get_genre(id, genres) for id in res["track2"]["word2vec"]["tracks"]]
q2_genre_bert = [get_genre(id, genres) for id in res["track2"]["bert"]["tracks"]]
q2_genre_random = [get_genre(id, genres) for id in res["track2"]["base_line"]["tracks"]]

#calculate genre diversity @10
res["track2"]["mfcc_stats"]["genre_diversity@10"] = gen_div_10(q2_genre_mfcc_stats, all_genres, N=10)
genre_div = res["track2"]["mfcc_stats"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["blf_correlation"]["genre_diversity@10"] = gen_div_10(q2_genre_blf_correlation,all_genres,N=10)
genre_div = res["track2"]["blf_correlation"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["ivec256"]["genre_diversity@10"] = gen_div_10(q2_genre_ivec256,all_genres,N=10)
genre_div = res["track2"]["ivec256"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["musicnn"]["genre_diversity@10"] = gen_div_10(q2_genre_musicnn,all_genres,N=10)
genre_div = res["track2"]["musicnn"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["tfidf"]["genre_diversity@10"] = gen_div_10(q2_genre_tfidf,all_genres,N=10)
genre_div = res["track2"]["tfidf"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["word2vec"]["genre_diversity@10"] = gen_div_10(q2_genre_word2vec,all_genres,N=10)
genre_div = res["track2"]["word2vec"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["bert"]["genre_diversity@10"] = gen_div_10(q2_genre_bert,all_genres,N=10)
genre_div = res["track2"]["bert"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["base_line"]["genre_diversity@10"] = gen_div_10(q2_genre_random,all_genres,N=10)
genre_div = res["track2"]["base_line"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")


In [ ]:
# calculate genre coverage@10 
res["track2"]["mfcc_stats"]["genre_coverage@10"] = gen_cov_10(res["track2"]["mfcc_stats"]["tracks"], genres)
genre_cov = res["track2"]["mfcc_stats"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["blf_correlation"]["genre_coverage@10"] = gen_cov_10(res["track2"]["blf_correlation"]["tracks"], genres)
genre_cov = res["track2"]["blf_correlation"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["ivec256"]["genre_coverage@10"] = gen_cov_10(res["track2"]["ivec256"]["tracks"], genres)
genre_cov = res["track2"]["ivec256"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["musicnn"]["genre_coverage@10"] = gen_cov_10(res["track2"]["musicnn"]["tracks"], genres)
genre_cov = res["track2"]["musicnn"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["tfidf"]["genre_coverage@10"] = gen_cov_10(res["track2"]["tfidf"]["tracks"], genres)
genre_cov = res["track2"]["tfidf"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["word2vec"]["genre_coverage@10"] = gen_cov_10(res["track2"]["word2vec"]["tracks"], genres)
genre_cov = res["track2"]["word2vec"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["bert"]["genre_coverage@10"] = gen_cov_10(res["track2"]["bert"]["tracks"], genres)
genre_cov = res["track2"]["bert"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["base_line"]["genre_coverage@10"] = gen_cov_10(res["track2"]["base_line"]["tracks"], genres)
genre_cov = res["track2"]["base_line"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

In [ ]:
# calculate ndcg@10 
res["track2"]["mfcc_stats"]["ndcg"] = ndcg_score(id_track2, res["track2"]["mfcc_stats"]["tracks"], genres)
ndcg = res["track2"]["mfcc_stats"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["blf_correlation"]["ndcg"] = ndcg_score(id_track2, res["track2"]["blf_correlation"]["tracks"], genres)
ndcg = res["track2"]["blf_correlation"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["ivec256"]["ndcg"] = ndcg_score(id_track2, res["track2"]["ivec256"]["tracks"], genres)
ndcg = res["track2"]["ivec256"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["musicnn"]["ndcg"] = ndcg_score(id_track2, res["track2"]["musicnn"]["tracks"], genres)
ndcg = res["track2"]["musicnn"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["tfidf"]["ndcg"] = ndcg_score(id_track2, res["track2"]["tfidf"]["tracks"], genres)
ndcg = res["track2"]["tfidf"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["word2vec"]["ndcg"] = ndcg_score(id_track2, res["track2"]["word2vec"]["tracks"], genres)
ndcg = res["track2"]["word2vec"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["bert"]["ndcg"] = ndcg_score(id_track2, res["track2"]["bert"]["tracks"], genres)
ndcg = res["track2"]["bert"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["base_line"]["ndcg"] = ndcg_score(id_track2, res["track2"]["base_line"]["tracks"], genres)
ndcg = res["track2"]["base_line"]["ndcg"]
print(f"ndcg: {ndcg}")


## Track 3
"Every Christmas" "Kelly Clarkson"

In [ ]:
name = 'Every Christmas'
artist = 'Kelly Clarkson'
id_track3 = get_id_from_info(name, artist, info)

In [ ]:
# retrieve tracks with random baseline 
res["track3"]["base_line"]["tracks"] = random_baseline(id=id_track3, info=info, N=10)

In [ ]:
# retrieve tracks with text based retrieval systems 
res["track3"]["tfidf"]["tracks"] = text_based(id=id_track3, repr=tfidf, N=10, sim_func=cos_sim)
res["track3"]["word2vec"]["tracks"] = text_based(id=id_track3, repr=word2vec, N=10, sim_func=cos_sim)
res["track3"]["bert"]["tracks"] = text_based(id=id_track3, repr=bert, N=10, sim_func=cos_sim)

In [ ]:
# retrieve tracks with audio based retrieval systems 
res["track3"]["blf_correlation"]["tracks"] = audio_based(id=id_track3, repr=blf_correlation, N=10, sim_func=cos_sim)
print("blf correlation Results:")
display_res(res["track3"]["blf_correlation"]["tracks"], info)

res["track3"]["ivec256"]["tracks"] = audio_based(id=id_track3, repr=ivec256, N=10, sim_func=cos_sim)
print("ivec256 Results:")
display_res(res["track3"]["ivec256"]["tracks"], info)

res["track3"]["mfcc_stats"]["tracks"] = audio_based(id=id_track3, repr=mfcc_stats, N=10, sim_func=cos_sim)
print("mfcc_stats Results:")
display_res(res["track3"]["mfcc_stats"]["tracks"], info)

res["track3"]["musicnn"]["tracks"] = audio_based(id=id_track3, repr=musicnn, N=10, sim_func=cos_sim)
print("musicnn Results:")
display_res(res["track3"]["musicnn"]["tracks"], info)


In [ ]:
#retrieve genres of the results for precision@10 & recall@10 calculation
q3_genres_mfcc_stats = get_genre_from_ids(res["track3"]["mfcc_stats"]["tracks"],genres)
q3_genres_blf_correlation = get_genre_from_ids(res["track3"]["blf_correlation"]["tracks"], genres)
q3_genres_ivec256 = get_genre_from_ids(res["track3"]["ivec256"]["tracks"], genres)
q3_genres_musicnn = get_genre_from_ids(res["track3"]["musicnn"]["tracks"], genres)
q3_genres_tfidf = get_genre_from_ids(res["track3"]["tfidf"]["tracks"], genres)
q3_genres_word2vec = get_genre_from_ids(res["track3"]["word2vec"]["tracks"],genres)
q3_genres_bert =  get_genre_from_ids(res["track3"]["bert"]["tracks"],genres)
q3_genres_base_line = get_genre_from_ids(res["track3"]["base_line"]["tracks"],genres)

In [ ]:
# calculate precision @10 for track3

#get query genre
query_genre3 = get_genre_from_query(id_track3, genres)

res["track3"]["mfcc_stats"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_mfcc_stats,10)
precision = res["track3"]["mfcc_stats"]["precision@10"]
print(f"precision@10: {precision}")

res["track3"]["blf_correlation"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_blf_correlation,10)
precision = res["track3"]["blf_correlation"]["precision@10"]
print(f"precision@10: {precision}")

res["track3"]["ivec256"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_ivec256,10)
precision = res["track3"]["ivec256"]["precision@10"] 
print(f"precision@10: {precision}")

res["track3"]["musicnn"]["precision@10"] = calculate_precision_at_k(query_genre3,q2_genres_musicnn,10)
precision = res["track3"]["musicnn"]["precision@10"]
print(f"precision@10: {precision}")

res["track3"]["tfidf"]["precision@10"]= calculate_precision_at_k(query_genre3,q3_genres_tfidf,10)
precision = res["track3"]["tfidf"]["precision@10"]
print(f"precision@10: {precision}")

res["track3"]["word2vec"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_word2vec,10)
precision = res["track3"]["word2vec"]["precision@10"]
print(f"precision@10: {precision}")

res["track3"]["bert"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_bert,10)
precision = res["track3"]["bert"]["precision@10"] 
print(f"precision@10: {precision}")

res["track3"]["base_line"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_base_line,10)
precision = res["track3"]["base_line"]["precision@10"]
print(f"precision@10: {precision}")

In [ ]:
#query track 3 recall (k=10) for all 8 Retrieval Systems

res["track3"]["mfcc_stats"]["recall@10"] = calculate_recall_at_k(query_genre3, q3_genres_mfcc_stats,genres_list,10)
recall = res["track3"]["mfcc_stats"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["blf_correlation"]["recall@10"] = calculate_recall_at_k(query_genre3, q3_genres_blf_correlation,genres_list,10)
recall = res["track3"]["blf_correlation"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["ivec256"]["recall@10"] = calculate_recall_at_k(query_genre3, q3_genres_ivec256,genres_list,10)
recall = res["track3"]["ivec256"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["musicnn"]["recall@10"] = calculate_recall_at_k(query_genre3, q3_genres_musicnn,genres_list,10)
recall = res["track3"]["musicnn"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["tfidf"]["recall@10"] = calculate_recall_at_k(query_genre3, q3_genres_tfidf,genres_list,10)
recall = res["track3"]["tfidf"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["word2vec"]["recall@10"]= calculate_recall_at_k(query_genre3, q3_genres_word2vec,genres_list,10)
recall = res["track3"]["word2vec"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["bert"]["recall@10"]= calculate_recall_at_k(query_genre3,q3_genres_bert,genres_list,10)
recall = res["track3"]["bert"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["base_line"]["recall@10"] = calculate_recall_at_k(query_genre3,q3_genres_base_line,genres_list,10)
recall = res["track3"]["base_line"]["recall@10"]
print(f"recall@10: {recall}")

In [ ]:
#retrieve 100 results for precision-recall plot (audio based) because k should be varied in the interval [1,100]
q3_100_mfcc_stats = get_genre_from_ids(audio_based(id=id_track3, repr=mfcc_stats, N=100, sim_func=cos_sim), genres)
q3_100_blf_correlation = get_genre_from_ids(audio_based(id=id_track3, repr=blf_correlation, N=100, sim_func=cos_sim), genres)
q3_100_ivec256 = get_genre_from_ids(audio_based(id=id_track3, repr=ivec256, N=100, sim_func=cos_sim), genres)
q3_100_musicnn = get_genre_from_ids(audio_based(id=id_track3, repr=musicnn, N=100, sim_func=cos_sim), genres)
q3_100_tfidf = get_genre_from_ids(text_based(id=id_track3, repr=tfidf, N=100, sim_func=cos_sim), genres)
q3_100_word2vec = get_genre_from_ids(text_based(id=id_track3, repr=word2vec, N=100, sim_func=cos_sim),genres)
q3_100_bert =  get_genre_from_ids(text_based(id=id_track3, repr=bert, N=100, sim_func=cos_sim),genres)
q3_100_base_line = get_genre_from_ids(random_baseline(id=id_track3, info=info, N=100),genres)

#plot precision recall curve for query track3 for all evaluated systems

system_data3 = {
    "audio mfcc stats": {"system_name": "Audio MFCC Stats","query_genre": query_genre3, "retrieved_genres": q3_100_mfcc_stats, "dataset_genres": genres_list},
    "audio blf correlation": {"system_name": "Audio BLF Correlation","query_genre": query_genre3, "retrieved_genres": q3_100_blf_correlation, "dataset_genres": genres_list},
    "audio ivec_256": {"system_name": "Audio iVec 256","query_genre": query_genre3, "retrieved_genres": q3_100_ivec256, "dataset_genres":genres_list},
    "audio muiscnn": {"system_name": "Audio Musicnn","query_genre": query_genre3, "retrieved_genres": q3_100_musicnn, "dataset_genres":genres_list},
    "text tf-idf": {"system_name": "Text TF-IDF","query_genre": query_genre3, "retrieved_genres": q3_100_tfidf, "dataset_genres":genres_list},
    "text word2vec": {"system_name": "Text Word2Vec","query_genre": query_genre3, "retrieved_genres": q3_100_word2vec, "dataset_genres":genres_list},
    "text bert": {"system_name": "Text BERT","query_genre": query_genre3, "retrieved_genres": q3_100_bert, "dataset_genres":genres_list},
    "text random": {"system_name": "Text Random Baseline","query_genre": query_genre3, "retrieved_genres": q3_100_base_line, "dataset_genres":genres_list},
}

plot_precision_recall_curve(system_data3)

In [ ]:
# calculate genre diversity@10
all_genres = list(set([item for id in genres['id'] for item in get_genre(id, genres)]))

#audio based
q3_genre_mfcc_stats = [get_genre(id, genres) for id in res["track3"]["mfcc_stats"]["tracks"]]
q3_genre_blf_correlation = [get_genre(id, genres) for id in res["track3"]["blf_correlation"]["tracks"]]
q3_genre_ivec256 = [get_genre(id, genres) for id in res["track3"]["ivec256"]["tracks"]]
q3_genre_musicnn = [get_genre(id, genres) for id in res["track3"]["musicnn"]["tracks"]]
#text based
q3_genre_tfidf = [get_genre(id, genres) for id in res["track3"]["tfidf"]["tracks"]]
q3_genre_word2vec = [get_genre(id, genres) for id in res["track3"]["word2vec"]["tracks"]]
q3_genre_bert = [get_genre(id, genres) for id in res["track3"]["bert"]["tracks"]]
q3_genre_random = [get_genre(id, genres) for id in res["track3"]["base_line"]["tracks"]]

#calculate genre diversity @10
res["track3"]["mfcc_stats"]["genre_diversity@10"] = gen_div_10(q3_genre_mfcc_stats, all_genres, N=10)
genre_div = res["track3"]["mfcc_stats"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["blf_correlation"]["genre_diversity@10"] = gen_div_10(q3_genre_blf_correlation,all_genres,N=10)
genre_div = res["track3"]["blf_correlation"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["ivec256"]["genre_diversity@10"] = gen_div_10(q3_genre_ivec256,all_genres,N=10)
genre_div = res["track3"]["ivec256"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["musicnn"]["genre_diversity@10"] = gen_div_10(q3_genre_musicnn,all_genres,N=10)
genre_div = res["track3"]["musicnn"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["tfidf"]["genre_diversity@10"] = gen_div_10(q3_genre_tfidf,all_genres,N=10)
genre_div = res["track3"]["tfidf"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["word2vec"]["genre_diversity@10"] = gen_div_10(q3_genre_word2vec,all_genres,N=10)
genre_div = res["track3"]["word2vec"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["bert"]["genre_diversity@10"] = gen_div_10(q3_genre_bert,all_genres,N=10)
genre_div = res["track3"]["bert"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["base_line"]["genre_diversity@10"] = gen_div_10(q3_genre_random,all_genres,N=10)
genre_div = res["track3"]["base_line"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")


In [ ]:
# calculate genre coverage@10 
res["track3"]["mfcc_stats"]["genre_coverage@10"] = gen_cov_10(res["track3"]["mfcc_stats"]["tracks"], genres)
genre_cov = res["track3"]["mfcc_stats"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["blf_correlation"]["genre_coverage@10"] = gen_cov_10(res["track3"]["blf_correlation"]["tracks"], genres)
genre_cov = res["track3"]["blf_correlation"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["ivec256"]["genre_coverage@10"] = gen_cov_10(res["track3"]["ivec256"]["tracks"], genres)
genre_cov = res["track3"]["ivec256"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["musicnn"]["genre_coverage@10"] = gen_cov_10(res["track3"]["musicnn"]["tracks"], genres)
genre_cov = res["track3"]["musicnn"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["tfidf"]["genre_coverage@10"] = gen_cov_10(res["track3"]["tfidf"]["tracks"], genres)
genre_cov = res["track3"]["tfidf"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["word2vec"]["genre_coverage@10"] = gen_cov_10(res["track3"]["word2vec"]["tracks"], genres)
genre_cov = res["track3"]["word2vec"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["bert"]["genre_coverage@10"] = gen_cov_10(res["track3"]["bert"]["tracks"], genres)
genre_cov = res["track3"]["bert"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["base_line"]["genre_coverage@10"] = gen_cov_10(res["track3"]["base_line"]["tracks"], genres)
genre_cov = res["track3"]["base_line"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

In [ ]:
# calculate ndcg@10 
res["track3"]["mfcc_stats"]["ndcg"] = ndcg_score(id_track3, res["track3"]["mfcc_stats"]["tracks"], genres)
ndcg = res["track3"]["mfcc_stats"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["blf_correlation"]["ndcg"] = ndcg_score(id_track3, res["track3"]["blf_correlation"]["tracks"], genres)
ndcg = res["track3"]["blf_correlation"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["ivec256"]["ndcg"] = ndcg_score(id_track3, res["track3"]["ivec256"]["tracks"], genres)
ndcg = res["track3"]["ivec256"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["musicnn"]["ndcg"] = ndcg_score(id_track3, res["track3"]["musicnn"]["tracks"], genres)
ndcg = res["track3"]["musicnn"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["tfidf"]["ndcg"] = ndcg_score(id_track3, res["track3"]["tfidf"]["tracks"], genres)
ndcg = res["track3"]["tfidf"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["word2vec"]["ndcg"] = ndcg_score(id_track3, res["track3"]["word2vec"]["tracks"], genres)
ndcg = res["track3"]["word2vec"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["bert"]["ndcg"] = ndcg_score(id_track3, res["track3"]["bert"]["tracks"], genres)
ndcg = res["track3"]["bert"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["base_line"]["ndcg"] = ndcg_score(id_track3, res["track3"]["base_line"]["tracks"], genres)
ndcg = res["track3"]["base_line"]["ndcg"]
print(f"ndcg: {ndcg}")

In [ ]:
# Apply the vectorized function with k = 2
df["recall_at_2"] = df.apply(lambda row: calculate_recall_at_k_vectorized([row["query_genre"]], [row["retrieved_genres"]], [row["dataset_genres"]], 10), axis=1)

In [ ]:
blf_correlation

In [ ]:
# Importieren Sie sklearn
from sklearn.metrics.pairwise import cosine_similarity
# Berechnen Sie die Kosinus-Ähnlichkeit aller IDs mit allen anderen IDs entlang der Reihen
similarity = cosine_similarity(blf_correlation[:][2:])

# Erhalten Sie nur die obere Dreiecksmatrix der Ähnlichkeitsmatrix
upper = np.triu(similarity)

In [ ]:
df


In [ ]:
df_merged = df.merge( genres,left_index=True, right_on='id',  how='inner')

In [ ]:
df_merged

In [ ]:
genres

In [ ]:
blf_correlation_matrix

In [ ]:
blf_correlation.shape

In [ ]:
 # Fill the lower triangular part with the same values as the upper triangular part
cos_sim_matrix = cos_sim_matrix + cos_sim_matrix.T - np.diag(cos_sim_matrix.diagonal())